### Forward Vector Wave Simulation using PINNs

This notebook demonstrates a forward simulation of vector waves using a Physics-Informed Neural Network (PINN) framework. In this notebook, we:

- Present the governing equations and explain each variable and dataset column.
- Describe the data processing steps.
- Define and train a neural network to solve the PDE system.
- Predict and visualize the solution.

---

#### Governing Equations

We aim to solve the following system (in nondimensional form):

\[
\begin{aligned}
\frac{d\tilde{u}}{dt} &= \tilde{v}, \\
\frac{d\tilde{v}}{dt} &= \kappa(v_0 t - \tilde{u}) - \alpha\Big(f_0 + a \ln(\tilde{v}) + b \ln(\tilde{\theta})\Big), \\
\frac{d\tilde{\theta}}{dt} &= -\tilde{v}\tilde{\theta}\ln(\tilde{v}\tilde{\theta}).
\end{aligned}
\]

The network is trained to predict the solutions for:

- \(\tilde{u}\): Slip
- \(\tilde{v}\): Slip rate
- \(\tilde{\theta}\): State variable

---

#### Explanation of Variables and Dataset Columns

- **Dataset Columns:**
  - **`Var1`**: Time
  - **`y1_1`**: \(\tilde{u}\) (slip)
  - **`y1_2`**: \(\tilde{v}\) (slip rate)
  - **`y1_3`**: \(\tilde{\theta}\) (state variable)

- **Model Parameters:**
  - \(\alpha = 9.81\)
  - \(\kappa = 0.25\)
  - \(v_0 = 1\)
  - \(f_0 = 0.2\)
  - \(a = 0.2\)
  - \(b = 0.3\)

- **Neural Network Architecture:**
  - **Input:** Time \(t\) (1 neuron)
  - **Hidden layers:** 6 layers with 64 neurons each (using the Tanh activation function)
  - **Output:** 3 neurons corresponding to \(\tilde{u}\), \(\tilde{v}\), and \(\tilde{\theta}\)

- **Training Data:**
  - 20,000 residual (collocation) points in the time domain \([0, 100]\).
  - Measurement (boundary/initial) data is obtained by interpolating the first 10,000 data points to 25 equidistant points.

---

#### Training Process Overview

1. **Data Import and Preprocessing:** Load the CSV dataset, select the first 10,000 data points, and interpolate to obtain 25 measurement points.
2. **Define Boundary Conditions:** Use the interpolated data for each variable to create boundary conditions.
3. **Define the PDE System:** Implement the governing equations using DeepXDE’s automatic differentiation.
4. **Neural Network Setup:** Create a feed-forward network and apply an output transform to enforce the initial conditions.
5. **Model Compilation and Training:** Compile the model using the Adam optimizer and train for 50,000 iterations.
6. **Prediction and Visualization:** Predict the solution over the time domain and compare it with the true data.


In [ ]:
!pip install deepxde

In [ ]:
import deepxde as dde
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deepxde.backend import tf


### Data Import and Processing

The dataset is assumed to be stored in a CSV file (e.g., `../Dataset/Forward_vector_wave.csv`) with the following columns:

- **`Var1`**: Time
- **`y1_1`**: \(\tilde{u}\) (slip)
- **`y1_2`**: \(\tilde{v}\) (slip rate)
- **`y1_3`**: \(\tilde{\theta}\) (state variable)

We extract the first 10,000 data points and interpolate these to obtain 25 equidistant measurement points.

In [ ]:
raw = pd.read_csv('../Dataset/Forward_vector_wave.csv')  # Adjust the file path as needed
raw = raw.iloc[0:10000]

# Extract columns
observe_t = raw['Var1']
u_ext = raw['y1_1']
v_ext = raw['y1_2']
theta_ext = raw['y1_3']

# Interpolate to get 25 equidistant measurement points
t_int = np.linspace(0, 100, 25)
u_int = np.interp(t_int, observe_t.values, u_ext.values)
v_int = np.interp(t_int, observe_t.values, v_ext.values)
theta_int = np.interp(t_int, observe_t.values, theta_ext.values)

# Plot the true data and measurement points
plt.figure()
plt.plot(observe_t, u_ext, label="u")
plt.plot(observe_t, v_ext, label="v")
plt.plot(observe_t, theta_ext, label="θ")
plt.scatter(t_int, u_int, color="red", label="Measured u")
plt.scatter(t_int, v_int, color="green", label="Measured v")
plt.scatter(t_int, theta_int, color="blue", label="Measured θ")
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.grid()
plt.show()


### Define Boundary/Initial Conditions

We now create boundary conditions for each variable using the measured (interpolated) data. These conditions will help guide the network to satisfy the initial and measurement constraints.

In [ ]:
t_int = t_int.reshape((-1, 1))
u_int = nu_int.reshape((-1, 1))
v_int = v_int.reshape((-1, 1))
theta_int = theta_int.reshape((-1, 1))

observe_y0 = dde.icbc.PointSetBC(t_int, nu_int, component=0)
observe_y1 = dde.icbc.PointSetBC(t_int, v_int, component=1)
observe_y2 = dde.icbc.PointSetBC(t_int, theta_int, component=2)


### Define the PDE/ODE System

We define the system of equations using DeepXDE’s automatic differentiation. To avoid issues with taking the logarithm of zero, we use `tf.clip_by_value` to restrict the domain of \(\tilde{v}\) and \(\tilde{\theta}\).

The equations implemented are:

\[
\begin{aligned}
\frac{d\tilde{u}}{dt} &= \tilde{v}, \\
\frac{d\tilde{v}}{dt} &= \kappa(v_0 t - \tilde{u}) - \alpha\Big(f_0 + a \ln(\tilde{v}) + b \ln(\tilde{\theta})\Big), \\
\frac{d\tilde{\theta}}{dt} &= -\tilde{v}\tilde{\theta}\ln(\tilde{v}\tilde{\theta}).
\end{aligned}
\]


In [ ]:
alpha = 9.81
kappa = 0.25
v0 = 1
f0 = 0.2
a = 0.2
b = 0.3

def ode_system(x, y):
    # Extract variables from the network output
    u = y[:, 0:1]
    v = y[:, 1:2]
    theta = y[:, 2:3]

    # Compute time derivatives using automatic differentiation
    du_t = dde.grad.jacobian(y, x, i=0)
    dv_t = dde.grad.jacobian(y, x, i=1)
    dtheta_t = dde.grad.jacobian(y, x, i=2)

    # Define the system of ODEs
    eq1 = du_t - tf.clip_by_value(v, 0, 13)
    eq2 = dv_t - kappa * (v0 * x - u) + alpha * (f0 + a * tf.math.log(tf.clip_by_value(v, 0, 13)) + b * tf.math.log(tf.clip_by_value(theta, 0, 11)))
    eq3 = dtheta_t + (tf.clip_by_value(v, 0, 13) * tf.clip_by_value(theta, 0, 11) * tf.math.log(tf.clip_by_value(v, 0, 13) * tf.clip_by_value(theta, 0, 11)))
    
    return [eq1, eq2, eq3]


### Define Geometry and Generate Residual Points

We define the time domain \([0, 100]\) as our geometry. Next, we generate 20,000 collocation (residual) points to evaluate the PDE residual.

In [ ]:
geom = dde.geometry.TimeDomain(0, 100)

# Create the data object for the PDE problem
# The list [observe_y0, observe_y1, observe_y2] specifies the measurement conditions
data = dde.data.PDE(geom, ode_system, [observe_y0, observe_y1, observe_y2], 20000, 0, num_test=3000)


### Neural Network Setup

We construct a feed-forward neural network with the following architecture:

- **Input:** 1 neuron (time) 
- **Hidden Layers:** 6 layers with 64 neurons each (using Tanh activation)
- **Output:** 3 neurons corresponding to \(\tilde{u}\), \(\tilde{v}\), and \(\tilde{\theta}\)

An output transform is applied to help enforce the initial conditions (e.g., setting \(\tilde{u}(0)=1\), \(\tilde{v}(0)=0.5\), \(\tilde{\theta}(0)=1\)).

In [ ]:
layer_size = [1] + [64] * 6 + [3]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

def output_transform(t, y):
    # Transform the network output to help enforce the initial conditions
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]
    y3 = y[:, 2:3]
    return tf.concat([y1 * tf.tanh(t) + 1, y2 * tf.tanh(t) + 0.5, y3 * tf.tanh(t) + 1], axis=1)

net.apply_output_transform(output_transform)


### Model Compilation and Training

We compile the model using the Adam optimizer (learning rate = 0.0001) with equal loss weights. The model is then trained for 50,000 iterations.

In [ ]:
model = dde.Model(data, net)
model.compile("adam", lr=0.0001, loss_weights=[1, 1, 1, 1, 1, 1])

losshistory, train_state = model.train(epochs=50000)


### Prediction and Visualization

We now predict the solution over the time domain \([0, 100]\) on a fine grid and compare the predictions with the true data. The true curves (from the dataset) are shown as solid lines, and the predicted curves are shown as dashed lines.

In [ ]:
plt.figure()
plt.xlabel("Time")
plt.ylabel("Value")

# Plot true measurements (interpolated)
plt.plot(observe_t, u_ext, color="black", label="True u")
plt.plot(observe_t, v_ext, color="blue", label="True v")
plt.plot(observe_t, theta_ext, color="brown", label=r'True $\theta$')

# Predict on a finer grid
t = np.linspace(0, 100, 10000).reshape(-1, 1)
sol_pred = model.predict(t)

u_pred = sol_pred[:, 0:1]
v_pred = sol_pred[:, 1:2]
theta_pred = sol_pred[:, 2:3]

plt.plot(t, u_pred, color="red", linestyle="dashed", label="Predicted u")
plt.plot(t, v_pred, color="orange", linestyle="dashed", label="Predicted v")
plt.plot(t, theta_pred, color="green", linestyle="dashed", label=r"Predicted $\theta$")
plt.legend()
plt.grid()
plt.show()


### Summary

In this notebook, we implemented a PINN to simulate forward vector wave propagation by solving a system of differential equations. We explained the governing equations and variables (including dataset columns), described the neural network architecture and training process, and compared the predicted solution with the true data.

Feel free to modify the equations, parameters, and network architecture as needed for your application.